In [1]:
import syft as sy

# Create duet session

In [2]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!

♫♫♫ > DUET LIVE STATUS  *  Objects: 0  Requests: 0   Messages: 1  Request Handlers: 0                                


# Import library

In [3]:
import torch
import numpy as np
import torch as th
import pandas as pd
from PIL import Image
import imageio.v2 as imageio
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Import metadata

In [4]:
df =pd.read_csv('HAM10000_metadata.csv')

In [5]:
df['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [5]:
target = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

# Create data subset

In [6]:
subset_metadata_a = df.sample(frac=0.2, random_state=42)   

In [7]:
num_samples_per_label_a = 300                                                 ## 300 sample for each class
balanced_sub_a = pd.DataFrame()
for label in target:
    label_metadata_a = subset_metadata_a[subset_metadata_a['dx'] == label]
    label_samples_a = label_metadata_a.sample(n=num_samples_per_label_a, random_state=42, replace = True)
    balanced_sub_a = pd.concat([balanced_sub_a, label_samples_a])

In [8]:
balanced_sub_a

,lesion_id,image_id,dx,dx_type,age,sex,localization
9707,HAM_0005931,ISIC_0031191,akiec,histo,60.0,male,lower extremity
9729,HAM_0007601,ISIC_0032437,akiec,histo,65.0,male,upper extremity
9924,HAM_0006434,ISIC_0030142,akiec,histo,60.0,male,lower extremity
9936,HAM_0006887,ISIC_0029500,akiec,histo,45.0,male,face
9888,HAM_0005231,ISIC_0025247,akiec,histo,70.0,male,lower extremity
...,...,...,...,...,...,...,...
2453,HAM_0001038,ISIC_0024706,vasc,consensus,50.0,male,trunk
2455,HAM_0001537,ISIC_0026467,vasc,consensus,55.0,male,lower extremity
2455,HAM_0001537,ISIC_0026467,vasc,consensus,55.0,male,lower extremity
2406,HAM_0003480,ISIC_0032890,vasc,consensus,70.0,female,lower extremity


In [9]:
sub_a_class_count = balanced_sub_a['dx'].value_counts()

In [10]:
sub_a_class_count

akiec    300
bcc      300
bkl      300
df       300
mel      300
nv       300
vasc     300
Name: dx, dtype: int64

In [11]:
size = (64, 64)
images_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_a = imageio.imread('HAM_data//HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_a).resize(size))
    images_sub_a.append(resized_img)

In [12]:
len(images_sub_a) 

2100

In [13]:
images_and_labels_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_b = imageio.imread('HAM_data/HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_b).resize(size))
    label = row['dx']
    images_and_labels_sub_a.append((resized_img, label))

In [14]:
# Convert list of tuples to NumPy arrays
images_sub_a, labels_sub_a = zip(*images_and_labels_sub_a)
images_sub_a = np.array(images_sub_a)
labels_sub_a = np.array(labels_sub_a)

# Print the shapes of the resulting arrays
print('Image shape:', images_sub_a.shape)
print('Label shape:', labels_sub_a.shape)

Image shape: (2100, 64, 64, 3)
Label shape: (2100,)


In [15]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_sub_a)

In [16]:
image_tensor = th.FloatTensor(images_sub_a.astype('float32'))
label_tensor = th.FloatTensor(labels_encoded.astype('float32'))

In [17]:
import torchvision.transforms as transforms
from PIL import Image

transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=0),    
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),    
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply the transformations to the data
x_train_transformed = []
for img in images_sub_a:
    img_pil = Image.fromarray(img)
    img_transformed = transform_train(img_pil)
    x_train_transformed.append(img_transformed)

In [18]:
x_train_transformed = torch.stack(x_train_transformed)

In [19]:
label_tensor.dtype

torch.float32

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_train_transformed, label_tensor, train_size=0.7, random_state=2019)

In [21]:
x_train = x_train.tag("x_train")
y_train = y_train.tag("y_train")
x_test = x_test.tag("x_test")
y_test = y_test.tag("y_test")

In [22]:
x_train_image = x_train.send(duet, searchable = True)
x_test_image = x_test.send(duet, searchable = True)
y_train_label = y_train.send(duet, searchable = True)
y_test_lable = y_test.send(duet, searchable = True)

`searchable` is deprecated please use `pointable` in future
`searchable` is deprecated please use `pointable` in future`searchable` is deprecated please use `pointable` in future
`searchable` is deprecated please use `pointable` in future


In [23]:
image_tensor.shape

torch.Size([2100, 64, 64, 3])

In [24]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: a25de6f4c0bf4aacb7f86f5691989e4f>,[x_train],,<class 'torch.Tensor'>
1,<UID: 1cb819713fc64832bf2464e6e70cc235>,[x_test],,<class 'torch.Tensor'>
2,<UID: beed35c6837f489182ce57ef8da327ab>,[y_train],,<class 'torch.Tensor'>
3,<UID: 8f8c2e4bd41a44ea8a1dc5bc5f18ec8f>,[y_test],,<class 'torch.Tensor'>


In [25]:
duet.requests.add_handler(action="accept")

Exception in callback Transaction.__retry()
handle: <TimerHandle when=1094367.406 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\nazmul_hossen\anaconda3\envs\pysyft1\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\nazmul_hossen\anaconda3\envs\pysyft1\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\nazmul_hossen\anaconda3\envs\pysyft1\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...03G\x95\xdfO'), ('172.16.1.66', 60713))>
